In [1]:
from langchain_community.graphs import Neo4jGraph

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(override=True)

# Access environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# Connection to Neo4j
graph = Neo4jGraph()

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI


prompt = """

you are an expert in routing queries against a vector-enabled graph data index and deciding whether it makes sense to perform a structure traversal upon the graph as the explicit structure of the graph is suited to answer the query or whether we will have to perform an unstructured search upon the index documents in the vector space, possibly prefiltering using the explicit graph structure.

Schema: {schema}
Question: {question}

"""

cypher_llm = AzureChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question, route it to the appropriate subgraph or perform an unstructured search.",
        ),
        ("human", prompt),
    ]
)

cypher_response = (
    RunnablePassthrough.assign(
        schema=lambda _: graph.get_schema,
    )
    | cypher_prompt
    | cypher_llm.bind(stop=["\nCypherResult:"])
    | StrOutputParser()
)

In [10]:
cypher_response.invoke({"question": "how many people are in the database?"})

'To answer the question "how many people are in the database?", it makes sense to perform a structured traversal of the graph. The `Person` node type directly represents individuals in the database, and you can count the instances of this node type.\n\n### Routing Decision:\n- **Action**: Perform a structured traversal.\n- **Node Type**: `Person`\n\n### Query:\nYou would execute a query to count the number of `Person` nodes in the graph. For example, in a Cypher-like query language, it would look something like this:\n\n```cypher\nMATCH (p:Person)\nRETURN COUNT(p) AS numberOfPeople\n```\n\nThis query will return the total number of `Person` nodes, which directly answers the question.'